# Anatomy of Pandas

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

## Basic Datatypes

### Series

In [ ]:
# Series are a (comparatively) thin wrapper around arrays

In [ ]:
# Get an array to work with

x = np.random.normal(size=(100,))

In [ ]:
# Turn this into a Series

simple_series = pd.Series(x)

# This contains the same data - but note the index column on the left
# All Series objects contain and Index in addition to their data
simple_series

In [ ]:
# By default, this index is an ordinal count (from 0), so the same as numpy/C indexes
# But, the similarity ends here!
# Pandas indexes are persistent, and will be subsetted along with data

simple_series[10:20]

In [ ]:
# Pandas indexes can be of (almost) any datatype
# The library includes some very useful and common cases - in particular, the DatetimeIndex
# There are many ways to construct these - we will use some of Pandas builtin tools for these examples

dti = pd.date_range('1 jun 2000', periods=100, freq='d')

# As you can see, this is a special class - it is not a Series or an Array, although it shares some features

dti

In [ ]:
# Rebuild our Series using the DatetimeIndex

date_series = pd.Series(x, index=dti)

date_series

In [ ]:
s.plot()

In [ ]:
# Now, indexing into the Series will use the Index data itself as index locations - not simply an integer index

start_date = dt.datetime(2000,7,15)

date_series[start_date]

In [ ]:
# DatetimeIndexes wrap the standard Python datetime library.  Get to the know this library, it makes working with indexing much easier!

end_date = start_date + dt.timedelta(10)

start_date, end_date

In [ ]:
# Note that unlike indexing with integers, indexing into a Series or DataFrame with a custom index class will
# select data inclusively (ie it is a closed interval on both ends) 

date_series[start_date:end_date]

In [ ]:
# You can still access the contents of a Series by (0-based) ordinal indexing, by using the iloc method
# Note that iloc indexes are also inclusive

date_series.iloc[0:10]

In [ ]:
# There is a similar method available that allows indexing by value rather than ordinal indexing
# Looks kind of pointless, since we can just use indexes...
# It will be important later!

date_series.loc[start_date:end_date]

In [ ]:
# The index of a Series is available as its own object - this will also be very useful later
date_series.iloc[0:10].index

In [ ]:
# Pandas has lots of convenience shortcuts, especially useful for interactive use

date_series['jul 2000':'aug 15 2000'].plot()

### DataFrame

A DataFrame can be thought of as a collection of Series with a shared Index

In [ ]:
# Let's construct a minimal DataFrame with just one Series - the date_series from above

df = pd.DataFrame({'x': date_series})

In [ ]:
# So far it doesn't contain anything additional to the Series data - with the exception of a Column name, 'x'

df

In [ ]:
# Columns are selected using standard indexing
# Selecting a single column will return the Series containing that column's data

df['x']

In [ ]:
# You can also select columns as if they were member variables of the DataFrame object

# Don't!
# Don't ever do this!
# This looks like it works fine...

df.x

In [ ]:
# ... but
# DataFrames have hundreds of methods and member variables
# The moment one of your columns shares a name with them, this happens...

bad_df = pd.DataFrame({'columns': [0,1,5,2]})
bad_df

In [ ]:
bad_df.columns

In [ ]:
# Because DataFrames are indexed 'column first', passing index values directly in will cause an error

df[start_date]

In [ ]:
# Or maybe they won't?

df['jul 2000']

In [ ]:
# Pandas is a big, complicated library with a lot of baggage and technical debt ("backwards compatibility")
# Wherever possible, use the least ambiguous methods you can
# In this case, that is the loc method (I told you it would be important)

df.loc[start_date]

In [ ]:
# Now, let's get a more complicated DataFrame from some real AuTuMN data

In [ ]:
from autumn.tools.runs import ManagedRun

In [ ]:
mr = ManagedRun("covid_19/hume/1633437782/f6ca627")

In [ ]:
param_df = mr.calibration.get_mcmc_params()

In [ ]:
# These are the parameters of a calibration run

param_df

In [ ]:
# As you can see, there are multiple columns in this run; you can access this programatically
# Columns is also an Index!  Just one that runs along a different axis

param_df.columns

In [ ]:
# Multiple columns can be selected at once.
# This is extremely useful in a programattic context, where lists can be generated in code and then used as arguments in indexing

param_df[['seasonal_force','vic_2021_seeding.seed_time','contact_rate']]

In [ ]:
# Let's get another DataFrame

mcmc_runs_df = mr.calibration.get_mcmc_runs()

In [ ]:
mcmc_runs_df

In [ ]:
# Boolean comparisons on Pandas objects produce boolean arrays, just like numpy

mcmc_runs_df['run'] > 500

In [ ]:
# You can use these to select subsets of Series or DataFrames

burned_df = mcmc_runs_df[mcmc_runs_df['run'] > 500]
burned_df

In [ ]:
# Take care to make sure your index matches the object you are indexing
# The following example will throw an exception in some versions of pandas, but just produce a warning in later versions
# Either way - don't do it!  Warnings exist for a reason, and if you see one, there is almost certainly a better way
# to write the code that produced it

burned_df[mcmc_runs_df['accept'] == 1] 

In [ ]:
# Same object use for comparison and indexing - no complaints

burned_df[burned_df['accept'] == 1] 

In [ ]:
# You can combine boolean indexes using boolean operators

mcmc_runs_df[(mcmc_runs_df['run'] > 500) & (mcmc_runs_df['accept'] == 1)]

In [ ]:
# Still.. that all seems a bit cumbersome
# OK, we're going to cheat a little here and use a custom function from the autumn library that makes life with pandas a little easier

from autumn.tools.utils.pandas import pdfilt

selected_runs = pdfilt(mcmc_runs_df, ["run > 500", "accept == 1"])
selected_runs

In [ ]:
# Now, we can access the index, and do something useful with it...

In [ ]:
selected_runs.index

In [ ]:
# Get the parameters from our params_df, using the index of our selected runs

param_df.loc[selected_runs.index]

## Pivots, Melts, MultiIndexing

In [ ]:
# Those DataFrames above looked nice.  A little too nice.  Is that even our data?

raw_params = mr.calibration.get_mcmc_params(raw=True)

raw_params

In [ ]:
# To reshape this DataFrame, we use the pivot_table method
# This needs to know which columns contain Index data, and which contain Column identifiers
# In this case, 'urun' has been handily filled in by combining run and chain in an earlier step, 
# so we can use this directly as an index

raw_params.pivot_table(index='urun',columns='name')

In [ ]:
# Hang on, that looks a bit weird - we've still got the chain and run columns in there, confusing matters... 
# Use drop to tidy things up

raw_params_urun = raw_params.drop(['chain','run'],axis='columns')
raw_params_urun.pivot_table(index='urun',columns='name')

In [ ]:
# An alternative, if we don't have a unique identifier, or more importantly want to retain access to both these 
# "dimensions", is to use a MultiIndex
# We'll drop 'urun', and build an index using both chain and run

midx_df = raw_params.drop('urun',axis='columns').pivot_table(index=['chain','run'], columns='name')
midx_df

In [ ]:
# MultiIndexes work a bit more like multidimensional arrays
# You can index by either subsetting on a single dimension...

midx_df.loc[6]

In [ ]:
# ... or by passing in multidimensional coordinates

midx_df.loc[6,13569]

In [ ]:
# Because columns are just Indexes on another axis, there can by column MultiIndexes too

In [ ]:
pbi = mr.powerbi.get_db()

In [ ]:
udf = pbi.get_uncertainty()

In [ ]:
udf

In [ ]:
# Finally - if you need to export this data, especially to CSV or a relational database, use melt
# This is the inverse of pivot_table

# It can require quite a lot of fine tuning, but is important to be aware of

udf.melt()

In [ ]:
melted = udf.melt(ignore_index=False)

In [ ]:
melted['date'] = melted.index

In [ ]:
melted